In [1]:
#find your file

import pandas as pd
import numpy as np
import glob
from functools import reduce
import ipywidgets 
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from PIL import Image

def clean_cols(df):
    for i, columns_old in enumerate(df.columns.levels):
        columns_new = np.where(columns_old.str.contains('Unnamed'), '', columns_old)
        df.rename(columns=dict(zip(columns_old, columns_new)), level=i, inplace=True)



In [2]:
#read it into a dataframe, use the right number of headers and index cols

df = pd.read_csv(r"C:\Users\Corey\learn-premierleague\data\AllSeasons\Match Data\epl_shots_AllSeasons_Season.csv", header=[0,1], index_col=[0,1,2,3,4])
sch = pd.read_csv(r"C:\Users\Corey\learn-premierleague\data\AllSeasons\NonMatchData\epl_schedule_AllSeasons_Season.csv")
clean_cols(df)
df = df.reset_index()
df.columns = df.columns.to_flat_index()
new_cols =['league',
 'season',
 'game',
 'team',
 'player',
 'minute',
 'xG',
 'PSxG',
 'outcome',
 'distance',
 'body_part',
 'notes',
 'SCA 1 player',
 'SCA 1 event',
 'SCA 2 player',
 'SCA 2 event']
df.columns = new_cols
sch=sch[['game','week','date']]
df = df.merge(sch, how='left', left_on='game',right_on='game')


In [3]:
df.head()


,league,season,game,team,player,minute,xG,PSxG,outcome,distance,body_part,notes,SCA 1 player,SCA 1 event,SCA 2 player,SCA 2 event,week,date
0,ENG-Premier League,1819,2018-08-10 Manchester Utd-Leicester City,Leicester City,Wilfred Ndidi,15,0.03,0.32,Saved,28,Right Foot,Deflected,Demarai Gray,Take-On,Ben Chilwell,Pass (Live),1,2018-08-10
1,ENG-Premier League,1819,2018-08-10 Manchester Utd-Leicester City,Leicester City,Wilfred Ndidi,25,0.03,NaN,Blocked,25,Right Foot,NaN,James Maddison,Pass (Live),Wilfred Ndidi,Pass (Live),1,2018-08-10
2,ENG-Premier League,1819,2018-08-10 Manchester Utd-Leicester City,Leicester City,James Maddison,29,0.07,0.02,Saved,13,Right Foot,NaN,Ricardo Pereira,Pass (Live),Wilfred Ndidi,Pass (Live),1,2018-08-10
3,ENG-Premier League,1819,2018-08-10 Manchester Utd-Leicester City,Leicester City,Ricardo Pereira,29,0.01,NaN,Blocked,33,Right Foot,NaN,NaN,NaN,NaN,NaN,1,2018-08-10
4,ENG-Premier League,1819,2018-08-10 Manchester Utd-Leicester City,Leicester City,James Maddison,32,0.06,NaN,Blocked,31,Right Foot,"Free kick, Deflected",Ben Chilwell,Pass (Dead),James Maddison,Fouled,1,2018-08-10


In [4]:
fd = pd.read_csv(r"G:\My Drive\Python - Reference\Fifa\raw_data\player_footed_data_2018to2022.csv",index_col='playerid')
fd = fd.drop(columns=['Id','Name'])
fd.columns = pd.MultiIndex.from_arrays([fd.columns, ['foot']])

fd.head()

,footed
,foot
playerid,
/en/players/5bc43860/Brenden-Aaronson,Right Foot
/en/players/f2bf1b0f/Che-Adams,Right Foot
/en/players/2b09d998/Tyler-Adams,Right Foot
/en/players/c81d773d/Tosin-Adarabioyo,Right Foot
/en/players/288e1e13/Nayef-Aguerd,Left Foot


In [5]:
def bpd(df):
    if df['body_part'] == "Head":
        val = "Head"
    elif df['body_part'] == 'Other':
        val = "Other"
    elif df['footed'] == 'No Data':
        val = "No Data"
    elif df['footed'] == df['body_part']:
        val = "Stong Foot"
    else:
        val = "Weak Foot"
    return val

df['PSxG'].fillna(0,inplace=True)
#df['strong_weak'] = df.apply(bpd, axis=1)     
df['psxg-xg'] = df['PSxG'] - df['xG']  

C:\Users\Corey\AppData\Local\Temp\ipykernel_8792\1654988887.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['PSxG'].fillna(0,inplace=True)


In [6]:
df.sample(7)

,league,season,game,team,player,minute,xG,PSxG,outcome,distance,body_part,notes,SCA 1 player,SCA 1 event,SCA 2 player,SCA 2 event,week,date,psxg-xg
37366,ENG-Premier League,2122,2022-05-12 Tottenham-Arsenal,Tottenham,Son Heung-min,47,0.09,0.30,Goal,14,Right Foot,NaN,NaN,NaN,NaN,NaN,22,2022-05-12,0.21
45331,ENG-Premier League,2223,2023-04-09 Leeds United-Crystal Palace,Leeds United,Luis Sinisterra,13,0.03,0.03,Saved,13,Head,NaN,Brenden Aaronson,Pass (Live),Jack Harrison,Pass (Live),30,2023-04-09,0.00
63198,ENG-Premier League,2425,2025-01-04 Manchester City-West Ham,West Ham,Mohammed Kudus,7,0.08,0.00,Off Target,6,Right Foot,NaN,Crysencio Summerville,Pass (Live),Crysencio Summerville,Take-On,20,2025-01-04,-0.08
52062,ENG-Premier League,2324,2023-12-17 West Ham-Wolves,Wolves,Mario Lemina,70,0.02,0.00,Off Target,29,Right Foot,NaN,Pablo Sarabia,Pass (Live),João Gomes,Pass (Live),17,2023-12-17,-0.02
37192,ENG-Premier League,2122,2022-05-08 Manchester City-Newcastle Utd,Manchester City,Oleksandr Zinchenko,68,0.03,0.04,Saved,20,Left Foot,NaN,Jack Grealish,Pass (Live),Raheem Sterling,Pass (Live),36,2022-05-08,0.01
20925,ENG-Premier League,2021,2020-11-21 Tottenham-Manchester City,Manchester City,Kevin De Bruyne,58,0.06,0.00,Blocked,17,Left Foot,NaN,Gabriel Jesus,Pass (Live),Rodri,Pass (Live),9,2020-11-21,-0.06
29356,ENG-Premier League,2122,2021-09-18 Burnley-Arsenal,Burnley,James Tarkowski,13,0.09,0.00,Off Target,7,Head,NaN,Dwight McNeil,Pass (Live),Ashley Westwood,Pass (Dead),5,2021-09-18,-0.09


In [7]:
wk = df.copy()

In [8]:
wkp = wk.groupby(['season','player'],as_index=False).aggregate(np.sum)
wkp = pd.DataFrame(wkp)
wkp
wf_xg_15 = wkp.sort_values(by=['season','xG'], ascending=[True,False]).groupby('season').head(15)
wf_xg_15 = wf_xg_15.pivot(index='player',columns='season',values='xG')
wf_xg_15 = wf_xg_15.unstack().reset_index(name='xG')
wf_xg_15


C:\Users\Corey\AppData\Local\Temp\ipykernel_8792\4289776690.py:1: FutureWarning: The provided callable <function sum at 0x0000023621F7E200> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  wkp = wk.groupby(['season','player'],as_index=False).aggregate(np.sum)


,season,player,xG
0,1819,Aleksandar Mitrović,13.35
1,1819,Alexander Isak,NaN
2,1819,Alexandre Lacazette,11.71
3,1819,Antoine Semenyo,NaN
4,1819,Ashley Barnes,12.28
...,...,...,...
359,2425,Tammy Abraham,NaN
360,2425,Teemu Pukki,NaN
361,2425,Timo Werner,NaN
362,2425,Yoane Wissa,8.32


In [9]:
wkp = wk.groupby(['season','player'],as_index=False).aggregate(np.sum)
wkp = pd.DataFrame(wkp)
wf_xg_15 = wkp.sort_values(by=['season','xG'], ascending=[True,False]).groupby('season').head(15)
wf_psxg_15 = wkp.sort_values(by=['season','PSxG'], ascending=[True,False]).groupby('season').head(15)
wf_psxg_xg_15 = wkp.sort_values(by=['season','psxg-xg'], ascending=[True,False]).groupby('season').head(15)

wf_xg_15 = wf_xg_15.pivot(index='player',columns='season',values='xG')
wf_xg_15 = wf_xg_15.unstack().reset_index(name='xG')
wf_xg_15['xG'].fillna(0,inplace=True)


wf_psxg_15 = wf_psxg_15.pivot(index='player',columns='season',values='PSxG')
wf_psxg_15 = wf_psxg_15.unstack().reset_index(name='PSxG')
wf_psxg_15['PSxG'].fillna(0,inplace=True)
wf_psxg_15

wf_psxg_xg_15 = wf_psxg_xg_15.pivot(index='player',columns='season',values='psxg-xg')
wf_psxg_xg_15 = wf_psxg_xg_15.unstack().reset_index(name='psxg-xg')
wf_psxg_xg_15['psxg-xg'].fillna(0,inplace=True)
wf_psxg_xg_15



C:\Users\Corey\AppData\Local\Temp\ipykernel_8792\473390318.py:1: FutureWarning: The provided callable <function sum at 0x0000023621F7E200> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  wkp = wk.groupby(['season','player'],as_index=False).aggregate(np.sum)
C:\Users\Corey\AppData\Local\Temp\ipykernel_8792\473390318.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  wf_xg_15['xG'].fillna(0,inplace=True

,season,player,psxg-xg
0,1819,Alex Iwobi,0.00
1,1819,Alexandre Lacazette,0.00
2,1819,Allan Saint-Maximin,0.00
3,1819,Amadou Onana,0.00
4,1819,Andrew Robertson,0.00
...,...,...,...
618,2425,Trevoh Chalobah,1.63
619,2425,Willian,0.00
620,2425,Youri Tielemans,0.00
621,2425,Yves Bissouma,0.00


In [23]:
fig = px.bar(
    x = wf_xg_15['player']
    ,y=wf_xg_15['xG']
    ,color=wf_xg_15['player']
    ,text=wf_xg_15['player']
    #,hover_data=[df['player'],df['Nation']]
    ,animation_frame = wf_xg_15['season']
    ,height=720
    ,width=1280
    ,title = f"xG from Weak Foot Shots - Top 15 Players by Season"
    ,labels={
        "x":"<b>Player</b>"
        ,"y":"<b>xG</b>"
        ,"color":"Player"
        #,"hover_data_0":"Player"
        #,"hover_data_1":"Nation"
    },
    range_y=[-0.2,25]
    )

fig.update_layout(
    showlegend=False,
    images=[dict(
        source=Image.open(rf"C:\Users\Corey\learn-premierleague\data\logos\EPL.png"),
        xref="paper", yref="paper",
        x=1, y=1.03,
        sizex=0.2, sizey=0.2,
        xanchor="right", yanchor="bottom"
      )],
    margin=dict(l=20, r=20, t=120, b=20),
    paper_bgcolor='lavender',
    title={
        'font':{
            'family':'Arial',
            'size' :25,
            'color':'#1C2C5b'
        }}#,
        #xaxis={'categoryorder':'total descending'}
)

#fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 2000
#fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 2000



fig.update_traces(textposition='inside')

fig.update_yaxes(showgrid=True, tickfont_family='Arial Black', tickfont_size=15, titlefont_size=20, titlefont_color='#1C2C5b', tickfont_color='#1C2C5b')
fig.update_xaxes(showgrid=True, tickfont_family='Arial Black', tickfont_size=15, titlefont_size=20, titlefont_color='#1C2C5b', tickfont_color='#1C2C5b',visible=False)

#fig.write_html(r"C:\Users\cmras\OneDrive - Poppulo\python training datasets\Vizzes\Corey\SCA_GCA_animated.html")
fig.show()

In [24]:
fig = px.bar(
    x = wf_psxg_15['player']
    ,y=wf_psxg_15['PSxG']
    ,color=wf_psxg_15['player']
    ,text=wf_psxg_15['player']
    #,hover_data=[df['player'],df['Nation']]
    ,animation_frame = wf_psxg_15['season']
    ,height=720
    ,width=1280
    ,title = f"PSxG from Weak Foot Shots - Top 15 Players by Season"
    ,labels={
        "x":"<b>Player</b>"
        ,"y":"<b>PSxG</b>"
        ,"color":"Player"
        #,"hover_data_0":"Player"
        #,"hover_data_1":"Nation"
    },
    range_y=[-0.2,25]
    )

fig.update_layout(
    showlegend=False,
    images=[dict(
        source=Image.open(rf"C:\Users\Corey\learn-premierleague\data\logos\EPL.png"),
        xref="paper", yref="paper",
        x=1, y=1.03,
        sizex=0.2, sizey=0.2,
        xanchor="right", yanchor="bottom"
      )],
    margin=dict(l=20, r=20, t=120, b=20),
    paper_bgcolor='lavender',
    title={
        'font':{
            'family':'Arial',
            'size' :25,
            'color':'#1C2C5b'
        }}#,
        #xaxis={'categoryorder':'total descending'}
)

#fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 2000
#fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 2000

fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            #x=0.7,
            #y=1.1,
            buttons=list([
                dict(
                    args=[None, dict(frame=dict(duration=2000), transition=dict(duration=2000))],
                    label="Play",
                    method="animate"
                ),
                dict(
                    args=[None, dict(frame=dict(duration=0), transition=dict(duration=0))],
                    label="Pause",
                    method="animate"
                )
            ]),
        )
    ]
)

fig.update_traces(textposition='inside')

fig.update_yaxes(showgrid=True, tickfont_family='Arial Black', tickfont_size=15, titlefont_size=20, titlefont_color='#1C2C5b', tickfont_color='#1C2C5b')
fig.update_xaxes(showgrid=True, tickfont_family='Arial Black', tickfont_size=15, titlefont_size=20, titlefont_color='#1C2C5b', tickfont_color='#1C2C5b',visible=False)

#fig.write_html(r"C:\Users\cmras\OneDrive - Poppulo\python training datasets\Vizzes\Corey\SCA_GCA_animated.html")
fig.show()

In [25]:
fig = px.bar(
    x = wf_psxg_xg_15['player']
    ,y=wf_psxg_xg_15['psxg-xg']
    ,color=wf_psxg_xg_15['player']
    ,text=wf_psxg_xg_15['player']
    #,hover_data=[df['player'],df['Nation']]
    ,animation_frame = wf_psxg_xg_15['season']
    ,height=720
    ,width=1280
    ,title = f"PSxG-xG from Weak Foot Shots - Top 15 Players by Season"
    ,labels={
        "x":"<b>Player</b>"
        ,"y":"<b>PSxG-xG</b>"
        ,"color":"Player"
        #,"hover_data_0":"Player"
        #,"hover_data_1":"Nation"
    },
    range_y=[-0.2,7]
    )

fig.update_layout(
    showlegend=False,
    images=[dict(
        source=Image.open(rf"C:\Users\Corey\learn-premierleague\data\logos\EPL.png"),
        xref="paper", yref="paper",
        x=1, y=1.03,
        sizex=0.2, sizey=0.2,
        xanchor="right", yanchor="bottom"
      )],
    margin=dict(l=20, r=20, t=120, b=20),
    paper_bgcolor='lavender',
    title={
        'font':{
            'family':'Arial',
            'size' :25,
            'color':'#1C2C5b'
        }}#,
        #xaxis={'categoryorder':'total descending'}
)

fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 2000
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 2000



fig.update_traces(textposition='inside')

fig.update_yaxes(showgrid=True, tickfont_family='Arial Black', tickfont_size=15, titlefont_size=20, titlefont_color='#1C2C5b', tickfont_color='#1C2C5b')
fig.update_xaxes(showgrid=True, tickfont_family='Arial Black', tickfont_size=15, titlefont_size=20, titlefont_color='#1C2C5b', tickfont_color='#1C2C5b',visible=False)

#fig.write_html(r"C:\Users\cmras\OneDrive - Poppulo\python training datasets\Vizzes\Corey\SCA_GCA_animated.html")
fig.show()

In [26]:
wk = wk.sort_values(by=['season','week','player'], ascending=[True,True,True])
wk['cumsum_xg'] = wk.groupby(['season','player'])['xG'].cumsum()
wk

,league,season,game,team,player,minute,xG,PSxG,outcome,distance,...,SCA 1 player,SCA 1 event,SCA 2 player,SCA 2 event,week,date,psxg-xg,cumsum_xg,cumsum_psxg,cumsum_psxg_xg
214,ENG-Premier League,1819,2018-08-12 Southampton-Burnley,Burnley,Aaron Lennon,11,0.44,0.76,Saved,12,...,Matthew Lowton,Pass (Live),James Tarkowski,Pass (Live),1,2018-08-12,0.32,0.44,0.76,0.32
163,ENG-Premier League,1819,2018-08-12 Arsenal-Manchester City,Arsenal,Aaron Ramsey,2,0.03,0.02,Saved,26,...,Sokratis Papastathopoulos,Pass (Live),NaN,NaN,1,2018-08-12,-0.01,0.03,0.02,-0.01
65,ENG-Premier League,1819,2018-08-11 Fulham-Crystal Palace,Fulham,Aboubakar Kamara,77,0.02,0.00,Off Target,29,...,Luciano Vietto,Pass (Live),Jean Seri,Pass (Live),1,2018-08-11,-0.02,0.02,0.00,-0.02
68,ENG-Premier League,1819,2018-08-11 Fulham-Crystal Palace,Fulham,Aboubakar Kamara,90+1,0.08,0.05,Saved,8,...,Aboubakar Kamara,Take-On,Luciano Vietto,Pass (Live),1,2018-08-11,-0.03,0.10,0.05,-0.05
55,ENG-Premier League,1819,2018-08-11 Fulham-Crystal Palace,Fulham,Aleksandar Mitrović,3,0.05,0.00,Blocked,23,...,Aleksandar Mitrović,Take-On,Jean Seri,Pass (Live),1,2018-08-11,-0.05,0.05,0.00,-0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63223,ENG-Premier League,2425,2025-01-04 Southampton-Brentford,Brentford,Yoane Wissa,60,0.40,0.00,Off Target,19,...,Kevin Schade,Pass (Live),Mathias Jensen,Pass (Live),20,2025-01-04,-0.40,7.96,8.13,0.17
63227,ENG-Premier League,2425,2025-01-04 Southampton-Brentford,Brentford,Yoane Wissa,72,0.03,0.07,Saved,15,...,Bryan Mbeumo,Pass (Live),Christian Nørgaard,Tackle,20,2025-01-04,0.04,7.99,8.20,0.21
63233,ENG-Premier League,2425,2025-01-04 Southampton-Brentford,Brentford,Yoane Wissa,90+4,0.33,0.86,Goal,12,...,NaN,NaN,NaN,NaN,20,2025-01-04,0.53,8.32,9.06,0.74
63095,ENG-Premier League,2425,2025-01-04 Aston Villa-Leicester City,Aston Villa,Youri Tielemans,37,0.02,0.00,Blocked,18,...,Boubacar Kamara,Pass (Live),Boubacar Kamara,Take-On,20,2025-01-04,-0.02,0.69,0.63,-0.06


In [27]:
wk = wk.sort_values(by=['season','week','player'], ascending=[True,True,True])
wk['cumsum_xg'] = wk.groupby(['season','player'])['xG'].cumsum()
wk['cumsum_psxg'] = wk.groupby(['season','player'])['PSxG'].cumsum()
wk['cumsum_psxg_xg'] = wk.groupby(['season','player'])['psxg-xg'].cumsum()
wk.loc[wk.player == 'Son Heung-min']

,league,season,game,team,player,minute,xG,PSxG,outcome,distance,...,SCA 1 player,SCA 1 event,SCA 2 player,SCA 2 event,week,date,psxg-xg,cumsum_xg,cumsum_psxg,cumsum_psxg_xg
1314,ENG-Premier League,1819,2018-09-22 Brighton-Tottenham,Tottenham,Son Heung-min,12,0.02,0.00,Off Target,29,...,Christian Eriksen,Pass (Live),Mousa Dembélé,Pass (Live),6,2018-09-22,-0.02,0.02,0.00,-0.02
1316,ENG-Premier League,1819,2018-09-22 Brighton-Tottenham,Tottenham,Son Heung-min,18,0.07,0.00,Blocked,17,...,Danny Rose,Pass (Live),Harry Kane,Pass (Live),6,2018-09-22,-0.07,0.09,0.00,-0.09
1318,ENG-Premier League,1819,2018-09-22 Brighton-Tottenham,Tottenham,Son Heung-min,29,0.05,0.00,Off Target,14,...,Harry Kane,Pass (Live),Christian Eriksen,Pass (Live),6,2018-09-22,-0.05,0.14,0.00,-0.14
1627,ENG-Premier League,1819,2018-09-29 Huddersfield-Tottenham,Tottenham,Son Heung-min,19,0.04,0.00,Off Target,19,...,Toby Alderweireld,Pass (Live),NaN,NaN,7,2018-09-29,-0.04,0.18,0.00,-0.18
1633,ENG-Premier League,1819,2018-09-29 Huddersfield-Tottenham,Tottenham,Son Heung-min,69,0.06,0.03,Saved,20,...,Harry Kane,Pass (Live),Son Heung-min,Pass (Live),7,2018-09-29,-0.03,0.24,0.03,-0.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62291,ENG-Premier League,2425,2024-12-15 Southampton-Tottenham,Tottenham,Son Heung-min,45,0.09,0.12,Saved,14,...,Dejan Kulusevski,Pass (Live),Pedro Porro,Pass (Live),16,2024-12-15,0.03,3.57,4.35,0.78
62571,ENG-Premier League,2425,2024-12-22 Tottenham-Liverpool,Tottenham,Son Heung-min,39,0.03,0.00,Off Target,18,...,James Maddison,Pass (Live),NaN,NaN,17,2024-12-22,-0.03,3.60,4.35,0.75
62723,ENG-Premier League,2425,2024-12-26 Nott'ham Forest-Tottenham,Tottenham,Son Heung-min,5,0.12,0.05,Saved,17,...,Dejan Kulusevski,Pass (Live),NaN,NaN,18,2024-12-26,-0.07,3.72,4.40,0.68
62729,ENG-Premier League,2425,2024-12-26 Nott'ham Forest-Tottenham,Tottenham,Son Heung-min,45+1,0.08,0.00,Off Target,23,...,Dominic Solanke,Fouled,Dejan Kulusevski,Pass (Live),18,2024-12-26,-0.08,3.80,4.40,0.60


In [28]:
s21_22 = wk.loc[wk.season == 2324]

######################### this was pasted in from clipboard #################################
week= pd.DataFrame(list(range(1,39)))
week.columns = ['week']
######################### this was pasted in from clipboard #################################

def player_stat(df, stat, player):
    df1 = df.loc[df.player == player]
    df1 = week.merge(df1, how='left', left_on='week',right_on='week')
    df1[stat] = df1[stat].fillna(0)
    df1['player'] = df1['player'].fillna(player)
    df1 = df1.groupby(['player','week'])[stat].sum()
    df1=df1.reset_index()
    df1[f'cumsum_{stat}'] = df1.groupby('player')[stat].cumsum()
    trace = go.Scatter(
        x = df1['week']
        ,y=df1[f'cumsum_{stat}']
        ,mode='lines'
        ,line=dict(width=1.5)
        ,name=player
        )
    return df1, trace

son, trace1 = player_stat(s21_22, 'xG','Son Heung-min')
saka, trace2 = player_stat(s21_22, 'xG','Bukayo Saka')
jota, trace3 = player_stat(s21_22, 'xG','Diogo Jota')
salah, trace4 = player_stat(s21_22, 'xG','Mohamed Salah')


frames = [dict(data=[dict(type='scatter',
                    x=son['week'][:k+1],
                    y=son['cumsum_xG'][:k+1]),
                    dict(type='scatter',
                    x=saka['week'][:k+1],
                    y=saka['cumsum_xG'][:k+1]),
                    dict(type='scatter',
                    x=jota['week'][:k+1],
                    y=jota['cumsum_xG'][:k+1]),
                    dict(type='scatter',
                    x=salah['week'][:k+1],
                    y=salah['cumsum_xG'][:k+1]),
                    ],
                traces = [0,1,2,3]
                )for k in range(1, len (week))]

layout = go.Layout(width=1280,
                   height=720,
                   showlegend=True,
                   hovermode='x unified',
                   title='Weak Foot Cumulative xG by Week',
                   updatemenus=[
                                {
                                    'buttons': [
                                        {
                                            'args': [None, {'frame': {'duration': 200, 'redraw': False},
                                                    'fromcurrent': True, 'transition': {'duration': 200}}],
                                            'label': 'Play',
                                            'method': 'animate'
                                        },
                                        {
                                            'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                                            'transition': {'duration': 0}}],
                                            'label': 'Pause',
                                            'method': 'animate'
                                        }
                                    ],
                                    'direction': 'left',
                                    'pad': {'r': 10, 't': 87},
                                    'showactive': False,
                                    'type': 'buttons',
                                    'x': 0.1,
                                    'xanchor': 'right',
                                    'y': 0,
                                    'yanchor': 'top'
                                }
                            ],     
                  )


layout.update(xaxis =dict(range=[0, 40], autorange=False),
              yaxis =dict(range=[0, 20], autorange=False))
fig = go.Figure(data=[trace1,trace2,trace3,trace4], frames=frames, layout=layout)
#fig.write_html(r"C:\Users\cmras\OneDrive - Poppulo\python training datasets\Vizzes\Corey\xG_Season_Player_animated.html")
fig.show()

